In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
import torch 
import torch.nn.functional as F
from sympy.vector import Laplacian
from torch.autograd.functional import hessian
from torch.distributions.multivariate_normal import MultivariateNormal
import seaborn as sns

**Instruções gerais:** Sua submissão <u>deve</u> conter: 
1. Um "ipynb" com seu código e as soluções dos problemas
2. Uma versão pdf do ipynb

Favor <u>não</u> enviar um .zip dos arquivos.
Caso você opte por resolver as questões de "papel e caneta" em um editor de $\LaTeX$ externo, o inclua no final da versão pdf do 'ipynb'--- submetendo um **<u>*único pdf*</u>**.

# Trabalho de casa 03: Regressão logística e inferência Bayesiana aproximada

O pedaço de código abaixo carrega o banco de dados 'breast cancer' e adiciona uma coluna de bias. Além disse, ele o particiona em treino e teste.

1. Implemente a estimativa de máximo a posteriori para um modelo de regressão logística com priori $\mathcal{N}(0, c I)$ com $c=100$ usando esse banco de dados;
2. Implemente a aproximação de Laplace para o mesmo modelo;
3. Implemente uma aproximação variacional usando uma Gaussiana diagonal e o truque da reparametrização;
4. Calcule a accuracy no teste para todas as opções acima --- no caso das 2 últimas, a prob predita é $\int_\theta p(y|x, \theta) q(\theta)$;
5. Para cada uma das 3 técnicas, plote um gráfico com a distribuição das entropias para as predições corretas e erradas (separadamente), use a função kdeplot da biblioteca seaborn.
6. Comente os resultados, incluindo uma comparação dos gráficos das entropias.

Explique sua implementação também! 

Para facilitar sua vida: use PyTorch, Adam para otimizar (é uma variação SGD) com lr=0.001, use o banco de treino inteiro ao invés de minibatchces, use `binary_cross_entropy_with_logits` para implementar a -log verossimilhança, use `torch.autograd.functional` para calcular a Hessiana. Você pode usar as bibliotecas importadas na primeira célula à vontade. Verifique a documentação de `binary_cross_entropy_with_logits` para garantir que a sua priori está implementada corretamente, preservando as proporções devidas. Use 10000 amostras das aproximações para calcular suas predições.

In [13]:
data =  load_breast_cancer()
N = len(data.data)
Ntrain = int(np.ceil(N*0.6))
perm = np.random.permutation(len(data.data))
X = torch.tensor(data.data).float()
X = torch.cat((X, torch.ones((X.shape[0], 1))), axis=1) 
y = torch.tensor(data.target).float()

Xtrain, ytrain = X[perm[:Ntrain]], y[perm[:Ntrain]]
Xtest, ytest = X[perm[Ntrain:]], y[perm[Ntrain:]]

In [14]:

    
def maximum_a_posteriori(X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    priori_var = 100
    learning_rate = 0.001

    d = X.shape[1]

    theta = torch.empty(d).normal_(std=priori_var ** 0.5).requires_grad_(True)
    optimizer = torch.optim.Adam([theta], lr=learning_rate)
    prior = MultivariateNormal(torch.zeros(d), priori_var * torch.eye(d))
    for _ in range(1000):
        optimizer.zero_grad()
        log_likelihood = F.binary_cross_entropy_with_logits(X @ theta, y, reduction='sum')
        log_prior = -prior.log_prob(theta)
        loss = log_likelihood + log_prior
        loss.backward()
        optimizer.step()
        if theta.grad.norm() < 1e-5:
            print("stop")
            break
        
    return theta
    

theta = maximum_a_posteriori(Xtrain, ytrain)

In [22]:
def laplacian(X: torch.Tensor, y: torch.Tensor):
    mode = maximum_a_posteriori(X,y)
    prior = MultivariateNormal(torch.zeros(X.shape[1]), 100 * torch.eye(X.shape[1]))
    log_posterior = lambda theta: F.binary_cross_entropy_with_logits(X @ theta, y, reduction="sum") - prior.log_prob(theta)
    hessian_posterior = hessian(log_posterior, mode)
    return MultivariateNormal(mode, torch.inverse(hessian_posterior))

def laplacian_pred(X, laplacian_posterior):
    samples = laplacian_posterior.sample((X.size(0),))
    y_pred = X.mm(samples.t())
    return y_pred

laplacian_approx = laplacian(Xtrain,ytrain)
y_pred_laplacian = laplacian_pred(X,laplacian_approx)

---

# Exercícios de "papel e caneta"

1. Derive a fórmula para a divergência KL entre duas distribuições Gaussianas univariadas, i.e., $D_\text{KL}\left(\mathcal{N}(\mu_1, \sigma_1^2) \| \mathcal{N}(\mu_2, \sigma_2^2)\right)$;

2. Suponha que $P$ é a família das distribuições categóricas com suporte em $\{1,\ldots, L\}$. Qual $p \in P$ possui maior entropia? 

3. Use a [desigualdade de Jensen](https://en.wikipedia.org/wiki/Jensen%27s_inequality) para mostrar que a divergência KL é não-negativa.

> **Dica:** A desigualdade de Jensen afirma que, se $\varphi$ é uma função convexa, então $\varphi(\mathbb{E}[X]) \leq \mathbb{E}[\varphi(X)]$.

4. Derive a aproximação de Laplace para a distribuição [Beta](https://en.wikipedia.org/wiki/Beta_distribution)($\alpha, \beta$). Mostre uma fórmula para valores genéricos $\alpha,\beta>1$ e a instancie para $\alpha=\beta=2$.

5. Derive a posteriori para o modelo Bayesiano com verossimilhança [Categórica](https://en.wikipedia.org/wiki/Categorical_distribution) e priori [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution), i.e.:
$$
\begin{align}{2}
y_1,\ldots, y_N &\sim Cat(\mathbf{\theta})\\
\mathbf{\theta} &\sim Dirichlet(\mathbf{\alpha})
\end{align}
$$
onde $\mathbf{\theta}$ e $\mathbf{\alpha}$ são vetores $L$-dimensionais.


### Questão 1
$$
\begin{align}
D_\text{KL}\left(\mathcal{N}(\mu_1, \sigma_1^2) \| \mathcal{N}(\mu_2, \sigma_2^2)\right) &= E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\log{\frac{\mathcal{N}(\mu_1,\sigma^2_1)}{\mathcal{N}(\mu_2,\sigma^2_2)}}\right]\\
&=$E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\log{\mathcal{N}(\mu_1,\sigma^2_1)}\right]-E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\log{\mathcal{N}(\mu_2,\sigma^2_2)}\right]\\
&=E_{\mathcal{N}(\mu_1,\sigma_1)}\left\{\log\left[{(2\pi\sigma^2_1)^{-\frac{1}{2}}\exp{\left[-\frac{1}{2}\left(\frac{x-\mu_1}{\sigma_1}\right)²\right]}}\right]\right\}-
E_{\mathcal{N}(\mu_1,\sigma_1)}\left\{\log\left[{(2\pi\sigma^2_2)^{-\frac{1}{2}}\exp{\left[-\frac{1}{2}\left(\frac{x-\mu_2}{\sigma_2}\right)²\right]}}\right]\right\}\\
&=-\frac{1}{2}\left\{E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\log(2\pi\sigma_1^2)\right]-E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\log(2\pi\sigma_2^2)\right]+
E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\left(\frac{x-\mu_1}{\sigma_1}\right)^2\right]-E_{\mathcal{N}(\mu_1,\sigma_1)}\left[\left(\frac{x-\mu_2}{\sigma_2}\right)^2\right]\right\}
\end{align}$$
